# Mid-Journey artistic idea prompt generator

- In this notebook, we will implement a brainstorming ideas function using OpenAI's GPT-3.5 model.
- The function will take features from an audience as input and generate a collection of diverse ideas that emotionally resonate with it.
- The ideas will be sent to Mid-Journey to generate artistic pictures that ultimately fit the target audience..

## Import libraries

In [4]:
import requests
import json
import os

from dotenv import load_dotenv
from pydantic import BaseModel
from typing import Literal, List

import autogen
from autogen import OpenAIWrapper, ConversableAgent, UserProxyAgent

## Load configurations

In [2]:
load_dotenv()

False

In [16]:
config_list = autogen.config_list_from_models(
    model_list=["gpt-3.5"],
)

client = OpenAIWrapper(config_list=config_list)

## Define classes

In [26]:
class Audience(BaseModel):
    """
    Represents the audience for brainstorming ideas.

    Attributes:
        age (str): The age of the audience.
        attributes (List[str]): List of attributes describing the audience.
    """
    age: str
    attributes: List[str]


class Idea(BaseModel):
    """
    Represents an idea for brainstorming.

    Attributes:
        title (str): The title of the idea.
        detail (str): Detailed description of the idea.
        style (str): The artistic style of the idea.
        procedure (str): The procedure to produce the idea.
    """
    title: str
    detail: str
    style: str
    procedure: str


class Ideas(BaseModel):
    """
    Represents a collection of brainstormed ideas.

    Attributes:
        ideas (List[Idea]): List of brainstormed ideas.
    """
    ideas: List[Idea]

## Define functions

In [27]:
def brainstorm_ideas(**kwargs) -> Ideas:
    """
    Brainstorms a diverse set of ideas that emotionally resonate with the audience.

    Args:
        **kwargs: Keyword arguments representing the audience.

    Returns:
        Ideas: A collection of brainstormed ideas.

    Raises:
        Exception: If the validation of the JSON response fails after multiple attempts.
    """
    audience = Audience.model_validate(kwargs)
    
    # Generate a conversation prompt to be sent to the OpenAI API
    conversation = client.create(
        messages=[
            {
                'role':'user',
                'content':f'''You are a master visual artist. Brainstorm a diverse set of ideas that emotionally resonate with the audience. Give each idea a title, specify the artistic style, a highly detailed description, and a procedure to produce it.
{{"audience":{kwargs}}}'''
            }
        ],
        model='gpt-35',
    ).choices[0].message.content
    
    num_try = 0
    
    schema = Ideas.model_json_schema()

    while num_try < 3:
        # Request a response from the OpenAI API based on the conversation prompt
        response = client.create(
            messages=[
                {
                    'role':'user',
                    'content':f'''You will return a JSON object according this json schema: {schema} \n\n Based on the following conversation:{conversation} \n\nReturn the JSON object only.''',
                }
            ]
        ).choices[0].message.content
        
        try:
            # Validate the response against the expected JSON schema
            return Ideas.model_validate_json(response)
        except:
            pass

    # If validation fails after multiple attempts, raise an exception
    raise Exception()

## Test the function and dump a brainstorm idea

In [28]:
audience = Audience(age='18-22', attributes=['energetic', 'undergraduate', 'aspiring computer scientist'])
brainstorm_ideas(**audience.model_dump())

Ideas(ideas=[Idea(title='Sparks of Creation', detail='This artwork aims to inspire and capture the energetic and creative essence of the audience. Using bold and vibrant colors, it presents a visual explosion of passion, imagination, and potential. The canvas is filled with energetic brushstrokes, overlapping shapes, and swirling lines, symbolizing the chaotic yet beautiful process of harnessing creativity. The artwork is dynamic and multi-dimensional, inviting the viewer to explore different layers and perspectives, just like the intricate world of computer science.', style='Abstract Expressionism', procedure='1. Start by preparing a large canvas and securing it on an easel or flat surface.\n2. Select a color palette that consists of bright, primary colors like red, blue, yellow, and their complimentary hues.\n3. Begin by applying a base coat of a bold primary color using a wide brush or palette knife, creating a vibrant background.\n4. Gradually add layers of contrasting and compleme

## Copy the content of "detail" into Mid-Journey and check for the results

This part is manually done on a private discord server. The output can be found in the folder "midjourney_outputs" of this repository.